# Create and Train our Transformer

### Load the Dataset

In [15]:
from datasets import load_dataset


# Load the dataset
train_dataset, test_dataset = load_dataset("dataset/PokedexDataset.py", split=["train", "test"])

Reusing dataset pokedex_dataset (C:\Users\Will Sumerfield\.cache\huggingface\datasets\pokedex_dataset\plain_text\1.0.0\3aa69cafe54b66fd5280fcf12671ebf466fe770853e5254944dd13cefd7a6698)


### Tokenize the Dataset

In [16]:
from transformers import AutoTokenizer


# Create the tokenizer instance
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

# Set the padding token to use the end of sentence token
tokenizer.pad_token = tokenizer.eos_token


def tokenize_data(data):
    """This function maps our dataset to a tokenized dataset"""
    tokenized_data = tokenizer(data['text'], max_length=55, padding='max_length', truncation=True)
    tokenized_data['labels'] = tokenized_data['input_ids']
    return tokenized_data


# Tokenize our datasets
tokenized_train_dataset = train_dataset.map(tokenize_data, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_data, batched=True)

### Import our Model

In [17]:
from transformers import AutoModelForCausalLM


# Get a pretrained instance of the model
model = AutoModelForCausalLM.from_pretrained("distilgpt2").to('cuda:0')

### Train the Model

In [18]:
from transformers import TrainingArguments
from transformers import Trainer


# Set the parameters for the trainer
training_args = TrainingArguments("model_checkpoints", per_device_train_batch_size=2, evaluation_strategy="epoch")

# Create a trainer with the provided arguments
trainer = Trainer(model=model,
                  args=training_args,
                  train_dataset=tokenized_train_dataset,
                  eval_dataset=tokenized_test_dataset,
                 )

In [19]:
# Train the Model
trainer.train()

Epoch,Training Loss,Validation Loss


TrainOutput(global_step=4608, training_loss=1.4535612298382654, metrics={'train_runtime': 381.8024, 'train_samples_per_second': 12.069, 'total_flos': 5838728417280.0, 'epoch': 3.0})

In [20]:
# Save the model
model.save_pretrained("models")

In [22]:
import shutil


# Delete the checkpoints
shutil.rmtree('model_checkpoints')


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'model_checkpoints'